In [3]:
%pylab inline
import requests, zipfile
import StringIO
import datetime
import pandas as pd
import numpy as np
from workflow.data import *
import pdb

Populating the interactive namespace from numpy and matplotlib


In [4]:
# find csv file for tripdata
year = 2015
month = 1
#csvPath = '{}{:02}-citibike-tripdata.csv'.format(year, month)
#df = pd.read_csv(basepath + csvPath, parse_dates = ['Start Time', 'Stop Time'])
trips = trip_data(year, month)
#df['trip_id'] = df.index.values

In [5]:
rebal_df = rebal_data(year,month)

In [6]:
#load stations.csv or create a stations df from tripdata
stations = pd.read_csv('stations.csv')
    
stations.head()

,id,docks,lat,long,name
0,72,39,40.767272,-73.993929,W 52 St & 11 Ave
1,79,33,40.719116,-74.006667,Franklin St & W Broadway
2,82,27,40.711174,-74.000165,St James Pl & Pearl St
3,83,62,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl
4,116,39,40.741776,-74.001497,W 17 St & 8 Ave


In [8]:
trips.head()

,duration,start_time,stop_time,start_id,start_name,start_lat,start_long,stop_id,stop_name,stop_lat,stop_long,bike_id,user_type,birth_year,gender
0,1346,2015-01-01 00:01:00,2015-01-01 00:24:00,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2
1,363,2015-01-01 00:02:00,2015-01-01 00:08:00,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1
2,346,2015-01-01 00:04:00,2015-01-01 00:10:00,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1
3,182,2015-01-01 00:04:00,2015-01-01 00:07:00,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1
4,969,2015-01-01 00:05:00,2015-01-01 00:21:00,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1


In [ ]:
df.start_time = pd.to_datetime(df.start_time)

In [ ]:
df.start_time[0]

## Interesting stats

In [ ]:
print "Total trips for {}-{:02}: {}".format(year, month, df.shape[0])

In [ ]:
sub_trips = df[df.user_type == 'Subscriber'].shape[0] / float(df.shape[0])
print "% subscribers: {:.3f}".format(100*sub_trips)

In [ ]:
male_trips = df[df.gender == 1].shape[0] / float(df.shape[0])
female_trips = df[df.gender == 2].shape[0] / float(df.shape[0])# add a column called trip_id within
print "% Male trips: {:.3f}".format(100*male_trips)
print "% Female trips: {:.3f}".format(100*female_trips)
print "% other trips: {:.3f}".format(100*(1 - male_trips - female_trips))

In [ ]:
# top 10 starting locations
top_starts = df.groupby(['start_name', 'start_lat', 'start_long']).count().duration.sort_values(ascending = False)
top_starts = top_starts.reset_index()
top_starts.head()

In [ ]:
# top 10 stopping locations
top_stops = df.groupby(['stop_name', 'stop_lat', 'stop_long']).count().duration.sort_values(ascending = False)
top_stops = top_stops.reset_index()
top_stops.head()

In [ ]:
def plot_top_stations(df, x_values, y_values):
    pl = df.plot(kind = 'scatter', x = x_values, y = y_values, 
                alpha = 0.3, figsize = (5,8))
    df.head(10).reset_index().plot(kind = 'scatter', x = x_values,
                 color = 'r', y = y_values, ax = pl)

plot_top_stations(top_starts, 'start_long', 'start_lat')
plot_top_stations(top_stops, 'stop_long', 'stop_lat')

## Rebalancing

In [ ]:
oneday = df[(df.start_time.dt.day == 1) & (df.start_time.dt.month == 3)]

In [ ]:
bike_list = df.bike_id.unique()

# collect rebal events for the first 100 bikes
df = df.sort_values(['bike_id', 'start_time'])
#cols = ['start_id','stop_id', 'start_time', 'stop_time', 'bike_id']
df1 = df[df.bike_id.isin(bike_list[:100])]

# track rebalancing events 
#(REMEMBER THAT REBAL EVENTS USE STOP_TIME AS THE STARTING TIME)

shift_cols = ['stop_id','stop_time', 'stop_long', 'stop_lat', 'stop_name']

def shift_cond(bike_df):
    bike_df[shift_cols] = bike_df[shift_cols].shift(1)
    return bike_df[bike_df.start_id != bike_df.stop_id]

rebal_df = oneday.groupby('bike_id').apply(shift_cond).dropna()
rebal_df.index = rebal_df.index.droplevel('bike_id')
rebal_df

In [6]:
rebal_df.head()

,duration,start_time,stop_time,start_id,start_name,start_lat,start_long,stop_id,stop_name,stop_lat,stop_long,bike_id,user_type,birth_year,gender,trip_id
0,1112,2015-01-02 06:37:00,2015-01-01 13:24:00,496,E 16 St & 5 Ave,40.737262,-73.992390,515.0,W 43 St & 10 Ave,40.760094,-73.994618,14529,Subscriber,1964.0,2,5715
1,925,2015-01-05 18:15:00,2015-01-05 02:30:00,456,E 53 St & Madison Ave,40.759711,-73.974023,173.0,Broadway & W 49 St,40.760647,-73.984427,14529,Subscriber,1962.0,1,40268
2,697,2015-01-07 06:53:00,2015-01-06 17:47:00,379,W 31 St & 7 Ave,40.749156,-73.991600,529.0,W 42 St & 8 Ave,40.757570,-73.990985,14529,Subscriber,1953.0,1,52693
3,349,2015-01-08 10:54:00,2015-01-07 07:05:00,468,Broadway & W 55 St,40.765265,-73.981923,497.0,E 17 St & Broadway,40.737050,-73.990093,14529,Subscriber,1979.0,1,64762
4,454,2015-01-12 06:55:00,2015-01-09 16:56:00,465,Broadway & W 41 St,40.755136,-73.986580,477.0,W 41 St & 8 Ave,40.756405,-73.990026,14529,Subscriber,1964.0,1,92747


In [ ]:
rebal_times = ((rebal_df.start_time - rebal_df.stop_time) / 2) + rebal_df.stop_time
rebal_spreads = (rebal_df.start_time - rebal_df.stop_time) / 2

In [ ]:
rebal_df.shape

## Calculate fluxes

In [ ]:
# select out top stations

In [4]:
#oneday = df[(df.start_time.dt.day == 1) & (df.start_time.dt.month) == 1]

In [7]:
def create_fluxes(df, id_key, time_key, fl_key):
    # id_key and time_key are start_id/stop_id and start_time/end_time
    # for normal trips, start is associated with an fl_key = 'flux_out',
    # and stop is associated with fl_key = 'flux_in'

    flux = df[[id_key, time_key]].groupby([df[time_key].dt.to_period('H'), id_key]).count()
    flux = flux[time_key].fillna(0).unstack(level=0).stack()
    flux = flux.reset_index().set_index(time_key)
    flux.index.name = 'date'
    return flux.rename(columns = {0:fl_key, id_key:'id'}).reset_index()

def merge_fluxes(flux_out, flux_in):
    return pd.merge(flux_out, flux_in, how='outer',
                    on = ['date', 'id']).fillna(0)

In [11]:
# create fluxes from normal trips
flux_out = create_fluxes(trips, 'start_id', 'start_time', 'flux_out')
flux_in = create_fluxes(trips, 'stop_id', 'stop_time', 'flux_in')

merged = merge_fluxes(flux_out, flux_in)

# create weekday column with values 1 if weekday and 0 if weekend
wkd_cond = lambda x: 1 if True else 0
merged['weekday'] = merged.date.dt.weekday.isin(range(5)).apply(wkd_cond)
merged['hour'] = merged.date.dt.hour

In [12]:
ages = year - trips.birth_year 
trips['ages'] = ages
trips.groupby([trips.start_time.dt.to_period('H'), ages]).mean()

duration    start_id  start_lat  start_long  \
start_time       birth_year                                                   
2015-01-01 00:00 19.0        1511.500000  367.000000  40.749167  -73.984615   
                 22.0         232.000000  293.000000  40.730287  -73.990765   
                 24.0         711.000000  305.000000  40.755255  -73.983097   
                 25.0         482.000000  148.000000  40.718562  -73.994982   
                 26.0         806.857143  393.428571  40.740412  -73.992036   
                 27.0         613.833333  345.666667  40.717893  -73.980247   
                 28.0         640.000000  294.000000  40.746037  -73.974315   
                 29.0         725.750000  383.875000  40.744946  -73.987384   
                 30.0        1039.833333  455.166667  40.741310  -73.995124   
                 31.0         267.000000  268.000000  40.719105  -73.999733   
                 32.0         781.000000  397.500000  40.742419  -73.990348   
                 33.0         283.500000  328.000000  40.703888  -73.979796   
                 34.0         443.000000  333.000000  40.716432  -73.972708   
                 35.0         578.750000  364.500000  40.734318  -73.983371   
                 36.0         425.571429  316.857143  40.724141  -73.992284   
                 38.0        1197.500000  417.000000  40.726174  -73.997852   
                 39.0         457.666667  277.333333  40.738896  -73.993839   
                 41.0         297.500000  429.500000  40.735675  -73.994086   
                 42.0        1111.000000  512.000000  40.750073  -73.998393   
                 43.0         394.666667  444.333333  40.742705  -73.992775   
                 44.0        1033.250000  827.500000  40.742458  -73.988601   
                 45.0         240.000000  268.500000  40.732003  -73.998571   
                 46.0         524.333333  458.666667  40.739018  -73.986983   
                 47.0         283.500000  849.500000  40.725836  -73.998308   
                 48.0         391.000000  359.000000  40.755103  -73.974987   
                 49.0         232.000000  449.000000  40.764618  -73.987895   
                 50.0         594.000000  323.000000  40.745372  -73.995086   
                 51.0         948.000000  329.250000  40.730406  -73.988000   
                 52.0         363.000000  434.000000  40.743174  -74.003664   
                 53.0         307.000000  474.000000  40.745168  -73.986831   
...                                  ...         ...        ...         ...   
2015-01-31 23:00 31.0         745.250000  377.500000  40.735810  -73.985497   
                 32.0         316.600000  446.400000  40.734608  -73.981457   
                 33.0         652.333333  361.666667  40.720047  -73.999555   
                 34.0         567.333333  271.666667  40.732197  -73.990638   
                 35.0         769.666667  421.000000  40.728334  -73.990130   
                 36.0         490.250000  300.250000  40.720239  -73.988611   
                 37.0         808.000000  441.500000  40.745240  -73.998384   
                 38.0         425.000000  352.000000  40.735176  -73.984796   
                 39.0        1210.000000  382.500000  40.750259  -73.993581   
                 41.0         760.250000  487.750000  40.744659  -73.986435   
                 42.0        1448.000000  341.666667  40.721435  -73.989400   
                 43.0         319.000000  453.000000  40.744751  -73.999154   
                 44.0         484.000000  369.000000  40.739734  -73.993222   
                 45.0         722.666667  337.333333  40.738438  -73.995797   
                 46.0         748.500000  356.000000  40.727836  -74.005239   
                 47.0         337.666667  484.333333  40.750601  -73.992295   
                 48.0         238.000000  395.000000  40.734075  -73.979928   
                 49.0        1244.500000  405.500000  40.755326  -73.979798   
   

In [56]:
rebal_df = rebal_data(year,month)
rebal_df = rebal_df[['start_time', 'stop_time', 'start_id', 'stop_id']]

In [57]:
def transform_times(df):
    # calculate approximate pickup and drop-off times for rebalancing trips
    t_start = df.start_time
    t_end = df.stop_time
    time_diff = t_start - t_end

    r_start = t_end + time_diff/3.
    r_end = t_end + time_diff*(2/3.)
    
    df['start_time'] = r_start
    df['stop_time'] = r_end
    return df.rename(columns = {'start_id':'stop_id', 'stop_id':'start_id'})

rebal_df = transform_times(rebal_df)

In [58]:
rflux_out = create_fluxes(rebal_df, 'start_id', 'start_time', 'rflux_out')
rflux_in = create_fluxes(rebal_df, 'stop_id', 'stop_time', 'rflux_in')
rmerged = merge_fluxes(rflux_out, rflux_in)

In [ ]:
merged

In [59]:
rmerged['net_flux'] = rmerged.rflux_in - rmerged.rflux_out
rmerged

,date,id,rflux_out,rflux_in,net_flux
0,2015-01-01 15:00,72.0,1.0,0.0,-1.0
1,2015-01-02 00:00,72.0,1.0,0.0,-1.0
2,2015-01-02 12:00,72.0,1.0,0.0,-1.0
3,2015-01-02 19:00,72.0,1.0,1.0,0.0
4,2015-01-02 20:00,72.0,1.0,0.0,-1.0
5,2015-01-05 08:00,72.0,1.0,0.0,-1.0
6,2015-01-05 11:00,72.0,1.0,0.0,-1.0
7,2015-01-06 08:00,72.0,1.0,0.0,-1.0
8,2015-01-08 16:00,72.0,1.0,0.0,-1.0
9,2015-01-09 01:00,72.0,1.0,0.0,-1.0


In [65]:
rmerged.sort_values('net_flux', ascending = False).head(50)

,date,id,rflux_out,rflux_in,net_flux
23282,2015-01-15 03:00,521.0,2.0,68.0,66.0
44432,2015-01-22 03:00,521.0,0.0,46.0,46.0
44377,2015-01-15 04:00,521.0,0.0,38.0,38.0
44327,2015-01-06 02:00,521.0,0.0,34.0,34.0
44423,2015-01-21 02:00,521.0,0.0,33.0,33.0
23148,2015-01-06 03:00,521.0,2.0,35.0,33.0
44431,2015-01-22 02:00,521.0,0.0,32.0,32.0
44424,2015-01-21 03:00,521.0,0.0,31.0,31.0
44376,2015-01-15 02:00,521.0,0.0,29.0,29.0
44433,2015-01-22 04:00,521.0,0.0,29.0,29.0


In [70]:
a = pd.merge(merged, rmerged, how = 'outer', on=['date','id']).fillna(0)

In [72]:
stations.head()

NameError: name 'stations' is not defined

In [71]:
a.head(20)

,date,id,flux_out,flux_in,weekday,hour,rflux_out,rflux_in,net_flux
0,2015-01-01 00:00,72.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2015-01-01 07:00,72.0,1.0,0.0,1.0,7.0,0.0,0.0,0.0
2,2015-01-01 11:00,72.0,1.0,0.0,1.0,11.0,0.0,0.0,0.0
3,2015-01-01 12:00,72.0,2.0,1.0,1.0,12.0,0.0,0.0,0.0
4,2015-01-01 15:00,72.0,1.0,0.0,1.0,15.0,1.0,0.0,-1.0
5,2015-01-01 17:00,72.0,2.0,2.0,1.0,17.0,0.0,0.0,0.0
6,2015-01-01 21:00,72.0,1.0,0.0,1.0,21.0,0.0,0.0,0.0
7,2015-01-01 22:00,72.0,1.0,2.0,1.0,22.0,0.0,0.0,0.0
8,2015-01-02 02:00,72.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0
9,2015-01-02 05:00,72.0,1.0,1.0,1.0,5.0,0.0,1.0,1.0


In [76]:
a.drop(['rflux_out', 'rflux_in'], axis=1)

,date,id,flux_out,flux_in,weekday,hour,net_flux
0,2015-01-01 00:00,72.0,1.0,0.0,1.0,0.0,0.0
1,2015-01-01 07:00,72.0,1.0,0.0,1.0,7.0,0.0
2,2015-01-01 11:00,72.0,1.0,0.0,1.0,11.0,0.0
3,2015-01-01 12:00,72.0,2.0,1.0,1.0,12.0,0.0
4,2015-01-01 15:00,72.0,1.0,0.0,1.0,15.0,-1.0
5,2015-01-01 17:00,72.0,2.0,2.0,1.0,17.0,0.0
6,2015-01-01 21:00,72.0,1.0,0.0,1.0,21.0,0.0
7,2015-01-01 22:00,72.0,1.0,2.0,1.0,22.0,0.0
8,2015-01-02 02:00,72.0,1.0,0.0,1.0,2.0,0.0
9,2015-01-02 05:00,72.0,1.0,1.0,1.0,5.0,1.0


In [80]:
a.head()

,date,id,flux_out,flux_in,weekday,hour,net_flux
0,2015-01-01 00:00,72.0,1.0,0.0,1.0,0.0,0.0
1,2015-01-01 07:00,72.0,1.0,0.0,1.0,7.0,0.0
2,2015-01-01 11:00,72.0,1.0,0.0,1.0,11.0,0.0
3,2015-01-01 12:00,72.0,2.0,1.0,1.0,12.0,0.0
4,2015-01-01 15:00,72.0,1.0,0.0,1.0,15.0,-1.0


In [83]:
stations = pd.read_csv('stations.csv')
stations.head()

aa = pd.merge(a, stations[['id', 'docks']], on = 'id')

,date,id,flux_out,flux_in,weekday,hour,net_flux,docks


In [ ]:
rflux_out = rebal_df[['stop_id', 'stop_time']].groupby([rebal_df.stop_time.dt.to_period('H'),'stop_id']).count()
rflux_in = rebal_df[['start_id', 'start_time']].groupby([rebal_df.start_time.dt.to_period('H'),'start_id']).count()

In [ ]:
rflux_out.head()

In [ ]:
t = rflux_out.fillna(0).stop_time.unstack(level=0).stack()
t = t.reset_index().set_index('stop_time')
t.rename(columns = {0:'rflux_out'}, inplace = True)
t.head()

In [ ]:
sample = flux_out.start_time.head(25).unstack(level=0).stack()
new_sample = sample.reset_index().set_index('start_time')
new_sample.columns

In [ ]:
# flux out is FROM start stations
flux_out = oneday.groupby('start_id').count().duration
flux_out.head(10)

In [ ]:
#flux in is TO stop stations
flux_in = oneday.groupby('stop_id').count().duration
flux_in.head(10)

In [ ]:
net_flux = flux_in - flux_out
net_flux.head()

In [ ]:
flux_out = df[['start_id', 'start_time']].groupby([df.start_time.dt.to_period('H'),'start_id']).count()
flux_in = df[['stop_id', 'stop_time']].groupby([df.stop_time.dt.to_period('H'),'stop_id']).count()

In [ ]:
flux_out.head()

In [ ]:
flux_out.unstack().fillna(0).start_time

In [ ]:
net_flux = flux_in.unstack().stop_time.fillna(0) - flux_out.unstack().start_time.fillna(0)

net_flux.shape

In [ ]:
from concat_data import create_fluxes

KeyboardInterrupt: 

In [ ]:
rebal_df[['start_id', 'start_time']].groupby([rebal_df.start_time.dt.to_period('2H'),'start_id']).count().unstack()

In [ ]:
rebal_df.start_id.astype(int).unique().shape

In [ ]:
rebal_df.stop_id.astype(int).unique().shape

In [ ]:
"""
#visualize rebalancing (with Bokeh)
import bokeh.plotting as bkp
import bokeh.models as bkm
import bokeh.charts as bkc
from bokeh.layouts import gridplot

N = 20  # number of top choices
station_lists = [0, 0]
station_lists[0] = xfers_in.groupby('start_id').apply(
                        lambda x: x.trip_id.count()).sort_values(ascending = False)[:N]
station_lists[1] = xfers_out.groupby('stop_id').apply(
                        lambda x: x.trip_id.count()).sort_values(ascending = False)[:N]

# output to notebook
bkp.output_notebook()

ps = [0, 0]  # initilized the list
source = [0, 0]  # initilized the list
colors = ['navy', 'red']  # ['In', 'Out']
labels = ['Out - In','In - Out']

for i, item in enumerate(['Transferred IN', 'Transferred OUT']):
    source[i] = bkp.ColumnDataSource(
    data = dict(station = list(map(str, station_lists[i].index)),
                count = station_lists[i].values,
                name = [stations.loc[x, 'name'] for x in station_lists[i].index],
                diff = [(-1)**(i+1)*station_count.loc[str(x), 'diff'] for x in station_lists[i].index]))
                        
    ps[i] = bkp.figure(plot_height = 300, plot_width = 350, toolbar_location = 'above',
                 tools = 'pan,box_zoom,reset,resize,save', 
                 x_range = list(map(str, station_lists[i].index)),
                 x_axis_label='Station id', y_axis_label = 'Counts', title = item)
    show = ps[i].vbar('station', top = 'count', width = 0.8, alpha = 0.5, line_alpha = 0.0,
               source = source[i], color = colors[i])
    ps[i].vbar('station', top = 'diff', width = 0.8, alpha = 1, line_alpha = 0.0,
               source = source[i], color = 'black')
    ps[i].xaxis.major_label_orientation = np.pi/4
    hover = bkm.HoverTool(tooltips=[('Station','@name'), 
                                    (item, '@count'),
                                    (labels[i],'@diff')], renderers=[show] )
    ps[i].add_tools(hover)
grid = gridplot([ps])

bkp.output_file('most_xferred_stations_'+str(year)+'.html')
bkp.show(grid) 
bkp.save(grid)
"""

In [ ]:
"""# find bikes by bike id
bike_ids = df.id.unique()
bike_trajectory = df[df.id == bike_ids[0]][['id', 'start_id', 'stop_id', 
                                            'start_time', 'stop_time',
                                           'start_long', 'start_lat',
                                           'stop_long', 'stop_lat']]

# find and link up bike trajectories to find rebalancing events
rebal_events = pd.DataFrame(columns = bike_trajectory.columns)
rebal_list = []

def find_rebal_events(traj, pre_or_post = 'pre'):
# find rebalancing events
    mask = (df.start_id.values[1:] != df.stop_id.values[:-1])
    if pre_or_post == 'pre':
        mask = np.hstack((False, mask))
    elif pre_or_post == 'post':
        masp = np.hstack((mask, False))

'''
    for bike_id in bike_ids[:10]:
        one_traj = df[df.id == bike_id][bike_trajectory.columns]
        #trajectories = pd.concat([trajectories, one_traj], ignore_index = True)

        # track rebalancing events by shifting down stop times (and locations)
        shift_cols = ['stop_id','stop_time', 'stop_long','stop_lat']
        one_traj[shift_cols] = one_traj[shift_cols].shift(1)
        
 

        # filter where start and stop station ids are unequal
        rebal = one_traj[one_traj.start_id != one_traj.stop_id]
        rebal = 
        rebal_list.append(rebal)
'''

find_rebal_events(df)
"""